In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import numpy as np

In [2]:
oldDF= pd.read_csv(r'C:\Users\Mohammed\Desktop\retail-DWH\data\online_retail_1.csv')
new_df= pd.read_csv(r'C:\Users\Mohammed\Desktop\retail-DWH\data\online_retail_1.csv')



In [7]:
def DimProduct(df= oldDF):
    dimproduct= df[['StockCode', 'Description']]\
        .drop_duplicates(subset=['StockCode', 'Description'])\
        .reset_index(drop=True)
    dimproduct['productKey']= range(1000, dimproduct['StockCode'].count()+1000, 1)
    dimproduct['name']= 'Product-'+dimproduct['StockCode']
    dimproduct['datesnapshot']= '20/2/2024'
    return dimproduct
old_data= DimProduct()
old_data.head()

,StockCode,Description,productKey,name,datesnapshot
0,22728,ALARM CLOCK BAKELIKE PINK,1000,Product-22728,20/2/2024
1,22727,ALARM CLOCK BAKELIKE RED,1001,Product-22727,20/2/2024
2,22726,ALARM CLOCK BAKELIKE GREEN,1002,Product-22726,20/2/2024
3,21724,PANDA AND BUNNIES STICKER SHEET,1003,Product-21724,20/2/2024
4,21883,STARS GIFT TAPE,1004,Product-21883,20/2/2024


In [8]:
def DimProduct(df= new_df):
    dimproduct= df[['StockCode', 'Description']]\
        .drop_duplicates(subset=['StockCode', 'Description'])\
        .reset_index(drop=True)
    dimproduct['productKey']= range(1000, dimproduct['StockCode'].count()+1000, 1)
    dimproduct['name']= 'Product-'+dimproduct['StockCode']
    dimproduct['datesnapshot']= '21/2/2024'
    return dimproduct
new_data= DimProduct()
new_data.head()

,StockCode,Description,productKey,name,datesnapshot
0,22728,ALARM CLOCK BAKELIKE PINK,1000,Product-22728,21/2/2024
1,22727,ALARM CLOCK BAKELIKE RED,1001,Product-22727,21/2/2024
2,22726,ALARM CLOCK BAKELIKE GREEN,1002,Product-22726,21/2/2024
3,21724,PANDA AND BUNNIES STICKER SHEET,1003,Product-21724,21/2/2024
4,21883,STARS GIFT TAPE,1004,Product-21883,21/2/2024


In [9]:
import datetime

def dimproduct_insert(new_df=cleaned_df_new, engine=db_engine):
    new_df['stockcode'] = new_df['stockcode'].astype(str)
    _date = datetime.datetime.now()
    new_date = _date + datetime.timedelta(days=2)
    snapshot_date = new_date.strftime('%Y-%m-%d %H:%M:%S')
    
    query = 'SELECT * FROM dimproduct_test'
    old_df = pd.read_sql(query, engine)
    old_stockcode = list(old_df['stockcode'])
    
    new_records = new_df[~new_df['stockcode'].isin(old_stockcode)]

    max_productid = 'SELECT MAX(productid) AS maxkey FROM dimproduct_test'
    maxid_df = pd.read_sql(max_productid, engine)
    maxid = int(maxid_df['maxkey'][0]) if not maxid_df.empty else 0
    print(maxid)
    if not new_records.empty:
        new_records = new_records[['stockcode', 'description']].drop_duplicates(subset=['stockcode', 'description']).reset_index(drop=True)
        new_records['productid'] = range(maxid + 1, len(new_records) + maxid + 1, 1)
        new_records['name'] = 'Product-' + new_records['stockcode']
        new_records['snapshot_date'] = snapshot_date
        new_records['snapshot_date']= pd.to_datetime(new_records['snapshot_date'])
        print(new_records.dtypes)
        try:
            new_records.to_sql(name='dimproduct_test', con=db_conn, if_exists='append', index=False)
            print("Insertion of new records done.")
        except Exception as e:
            print('Got error in insertion.', e)
    else:
        print("There aren't new data to be inserted.")
dimproduct_insert()

  StockCode                      Description  productKey           name  \
0     22728        ALARM CLOCK BAKELIKE PINK        1000  Product-22728   
1     22727         ALARM CLOCK BAKELIKE RED        1001  Product-22727   
2     22726       ALARM CLOCK BAKELIKE GREEN        1002  Product-22726   
3     21724  PANDA AND BUNNIES STICKER SHEET        1003  Product-21724   
4     21883                  STARS GIFT TAPE        1004  Product-21883   

  datesnapshot  
0   2024-02-21  
1   2024-02-21  
2   2024-02-21  
3   2024-02-21  
4   2024-02-21  


In [ ]:
import datetime

def dimproduct_update(new_df=cleaned_df_new, engine=db_engine):
    new_df['stockcode'] = new_df['stockcode'].astype(str)
    _date = datetime.datetime.now()
    new_date = _date + datetime.timedelta(days=2)
    snapshot_date = new_date.strftime('%Y-%m-%d %H:%M:%S')

    old_df = pd.read_sql('SELECT * FROM dimproduct_test', engine)
    old_stockcode = set(old_df['stockcode'])
    
    updated_records = new_df[new_df['stockcode'].isin(old_stockcode)]

    if not updated_records.empty:
        updated_records = updated_records[['stockcode', 'description']].drop_duplicates(subset=['stockcode', 'description']).reset_index(drop=True)
        print(updated_records.head())
        with engine.cursor() as cursor:
            for index, row in updated_records.iterrows():
                update_stmt = """
                    UPDATE dimproduct_test
                    SET description = %s, snapshot_date = %s
                    WHERE stockcode = %s
                """
                cursor.execute(update_stmt, (row["description"], snapshot_date, row["stockcode"]))
        
        engine.commit()
        print("dimproduct table updated successfully.")
        print(updated_records)
    else:
        print("No records to update.")
dimproduct_update()